In [12]:
import pandas as pd
import joblib
from predictive_model import FOMCPredictor
import numpy as np

def test_model():
    # Load the test data
    df = pd.read_csv('merged_fomc_market_data.csv')
    
    # Load the trained model and vectorizer
    model = joblib.load('fomc_predictor.joblib')
    tfidf = joblib.load('fomc_tfidf.joblib')
    
    # Create predictor instance
    predictor = FOMCPredictor()
    predictor.model = model
    predictor.tfidf = tfidf
    
    # Create results dataframe
    results = []
    
    # Test each speech
    for _, row in df.iterrows():
        prediction, probabilities = predictor.predict(row['clean_text'], row['speech_length'])
        
        results.append({
            'date': row['date'],
            'actual_market_reaction': row['market_reaction_up_or_down'],
            'predicted_market_reaction': prediction,
            'prediction_confidence': max(probabilities),
            'correct_prediction': prediction == row['market_reaction_up_or_down']
        })
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    
    # Calculate overall accuracy
    accuracy = (results_df['correct_prediction'].sum() / len(results_df)) * 100
    
    # Print summary statistics
    print(f"\nOverall Accuracy: {accuracy:.2f}%")
    print("\nPrediction Results by Market Direction:")
    print(results_df.groupby(['actual_market_reaction', 'predicted_market_reaction']).size().unstack())
    
    # Save results to CSV
    results_df.to_csv('prediction_results.csv', index=False)
    print("\nDetailed results saved to 'prediction_results.csv'")

if __name__ == '__main__':
    test_model()


[nltk_data] Downloading package punkt to /home/si295/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/si295/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/si295/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!



Overall Accuracy: 88.89%

Prediction Results by Market Direction:
predicted_market_reaction  Down   Up
actual_market_reaction              
Down                        8.0  NaN
Up                          2.0  8.0

Detailed results saved to 'prediction_results.csv'
